In [4]:
import cv2
import os 
import pickle
import mediapipe as mp

In [2]:
with open('trained_models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('trained_models/random_forest_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

confidence = 0.5
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=confidence)

I0000 00:00:1711167228.361744    4288 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1711167228.366760    4544 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
def get_predicted_gesture_image(image, hands):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    results = hands.process(image)

    image.flags.writeable = True
    
    #converting the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    both_hand_landmarks = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = []
            for landmark in hand_landmarks.landmark:
                # Extract x, y coordinates (relative to image dimensions)
                x = landmark.x
                y = landmark.y
                # Append coordinates to the list
                landmarks.append((x, y))
            both_hand_landmarks.append(landmarks)
        
        if len(both_hand_landmarks) == 1:
            both_hand_landmarks.append([(0, 0)] * len(both_hand_landmarks[0]))
        values = list(np.array(both_hand_landmarks).flatten())
        values = scaler.transform([values])
        predicted = loaded_model.predict(values)
        cv2.rectangle(image, (0,0), (160, 60), (245, 90, 16), -1)
        # Displaying Class
        cv2.putText(image, 'Predicted Gesture'
                    , (20,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(predicted[0])
                    , (20,45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return image


In [6]:
testcases_folder = 'testcases'
testcase_results_folder = 'testcase_results'

for filename in os.listdir(testcases_folder):
    if filename.startswith('test_case_') and filename.endswith('.jpg'):
        # Read image
        image_path = os.path.join(testcases_folder, filename)
        image = cv2.imread(image_path)

        # Process image
        processed_image = get_predicted_gesture_image(image, hands)

        # Save processed image
        output_filename = filename.replace('test_case_', 'testcase_output_')
        output_path = os.path.join(testcase_results_folder, output_filename)
        cv2.imwrite(output_path, processed_image)

        print(f"Processed and saved: {output_path}")

/home/karthik/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/karthik/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processed and saved: testcase_results/testcase_output_3.jpg
Processed and saved: testcase_results/testcase_output_1.jpg
Processed and saved: testcase_results/testcase_output_2.jpg
Processed and saved: testcase_results/testcase_output_4.jpg


/home/karthik/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/karthik/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
